# Non Planar Reconstruction (NPR) and Cross-Sectional View of a Vessel

aseresti@github.com

The goal of this script is to take the centerline of the vessel in .pth format (SimVascular Path files) and extract the NPR and cross-sections of the vessel at a given point.


### Import frameworks and define necessary functions

In [22]:
import vtk
import xml.etree.ElementTree as ET
import os

def ReadVTPFile(FileName):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader

def CutPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    return Slice.GetOutput()

def ReadVTIFile(FileName):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader.GetOutput()

def WriteVTPFile(FileName,Data):
    writer=vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(FileName)
    writer.SetInputData(Data)
    writer.Update()

def SphereClip(volume_image,center,radius):
    sphere = vtk.vtkSphere()
    sphere.SetCenter(center)
    sphere.SetRadius(radius)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume_image)
    clipper.SetClipFunction(sphere)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def PlaneClip(volume,center, a):
    box = vtk.vtkBox()
    box.SetBounds(
        center[0] - a, center[0] + a,
        center[1] - a, center[1] + a,
        center[2] - a, center[2] + a
    )

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume)
    clipper.SetClipFunction(box)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def gradient_filter(vtk_image):
    gradient_filter = vtk.vtkImageGradient()
    gradient_filter.SetInputData(vtk_image)
    gradient_filter.SetDimensionality(3)
    gradient_filter.Update()

    return gradient_filter.GetOutput()
    
def define_borders(gradient_image):
    magnitude_filter = vtk.vtkImageMagnitude()
    magnitude_filter.SetInputData(gradient_image)
    magnitude_filter.Update()
    
    return magnitude_filter.GetOutput()

def ExtractSurface(UnstrcturedGrid):
    geometry_filter = vtk.vtkGeometryFilter()
    geometry_filter.SetInputData(UnstrcturedGrid)
    geometry_filter.Update()

    return geometry_filter.GetOutput()


### Enter the path to the SimVascular pathfile of the vessel and VTI image

In [2]:
pathline_file = "/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU07A/SimVascular_Derrick/Paths/L_LAD_0.pth"
ImagePath = "/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU07A/SimVascular_Derrick/Images/S07ap7.vti"

### Read the pathline

In [3]:
with open(pathline_file, "r") as path:
    #path.readlines()
    tree = ET.parse(path)
root = tree.getroot()


direction_points = []
for direction_point in root.findall(".//path_point/tangent"):
    x = float(direction_point.attrib['x'])
    y = float(direction_point.attrib['y'])
    z = float(direction_point.attrib['z'])
    direction_points.append((x,y,z))

path_points = []
for path_point in root.findall(".//path_point/pos"):
    x = float(path_point.attrib['x'])
    y = float(path_point.attrib['y'])
    z = float(path_point.attrib['z'])
    path_points.append((x,y,z))

path_normals = []
for normal in root.findall(".//path_point/rotation"):
    x = float(normal.attrib['x'])
    y = float(normal.attrib['y'])
    z = float(normal.attrib['z'])
    path_normals.append((x,y,z))


### Read the image and Extract the gradient of the CTA

In [4]:
Volume = ReadVTIFile(ImagePath)
Volume_Gradient = define_borders(gradient_filter(Volume))

### Transform the vessel from 3D to 2D

 - LAD: in XY plane, flatten Z, note: for proximal Left main and LAD aroun the heart apex doesn't work well.

In [5]:
projection_matrix_XY = (   #Flatten Z
    1, 0, 0, 0,
    0, 1, 0, 0,
    0, 0, 0, 0,
    0, 0, 0, 1
)

projection_matrix_XZ = (   #Flatten Y
    1, 0, 0, 0,
    0, 0, 0, 0,
    0, 0, 1, 0,
    0, 0, 0, 1
)

projection_matrix_YZ = (   #Flatten X
    0, 0, 0, 0,
    0, 1, 0, 0,
    0, 0, 1, 0,
    0, 0, 0, 1
)



### Select an appropriate projection matrix based on the orientation of the vessel

In [6]:
projection_matrix = projection_matrix_XY

transform = vtk.vtkTransform()
transform.SetMatrix(projection_matrix)
transform.Update()

def Transform3Dto2D(section):
    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(section)
    transform_filter.SetTransform(transform)
    transform_filter.Update()

### Extract the vessel NPR

In [7]:
NPoints = len(path_points)
print("the number of points in the centerline is:", NPoints)

append_filter_npr3d = vtk.vtkAppendPolyData()
append_filter_npr2d = vtk.vtkAppendPolyData()
for i in range(5,NPoints, 5):
    p = path_points[i]
    p_ = path_points[i-5]
    a = (((p[0]-p_[0])**2 + (p[1]-p_[1])**2 + (p[2]-p_[2])**2)**0.5)/2
    npr3d_ = CutPlane(PlaneClip(Volume,path_points[i],a),path_points[i], path_normals[i])
    npr2d_ = Transform3Dto2D(npr3d_)

    append_filter_npr3d.AddInputData(npr3d_)
    append_filter_npr2d.AddInputData(npr2d_)

append_filter_npr2d.Update()
append_filter_npr3d.Update()

the number of points in the centerline is: 106


2025-02-07 13:21:27.258 (2021.336s) [          57BDD7]vtkDemandDrivenPipeline:663    ERR| vtkCompositeDataPipeline (0x7fea60b30f20): Input port 0 of algorithm vtkAppendPolyData (0x7fea97f26a70) has 0 connections but is not optional.


### Extract the Vessel Cross-section

Pick a point number in the range of the number of points contained in the centerline of the vessel. Adjust the number based on the location of the desired cross-section (proximal/mid/distal).

In [15]:
cs_loc = 50
CrossSection = CutPlane(PlaneClip(Volume,path_points[cs_loc],0.5),path_points[cs_loc],direction_points[cs_loc])
CrossSectionGrd = CutPlane(PlaneClip(Volume_Gradient,path_points[cs_loc],0.5),path_points[cs_loc],direction_points[cs_loc])

### Save Files

In [17]:
OutputFolder = os.path.dirname(pathline_file)
VesselName = os.path.splitext(os.path.basename(pathline_file))[0]
OutputFolder = os.path.join(OutputFolder,VesselName)

os.system(f"mkdir {OutputFolder}")

npr3d_FileName = f"{VesselName}_NPR3D.vtp"
npr2d_FileName = f"{VesselName}_NPR2D.vtp"

WriteVTPFile(os.path.join(OutputFolder,npr3d_FileName), append_filter_npr3d.GetOutput())
WriteVTPFile(os.path.join(OutputFolder,npr2d_FileName), append_filter_npr2d.GetOutput())


mkdir: /Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU07A/SimVascular_Derrick/Paths/L_LAD_0: File exists


In [18]:

CS_FileName = f"{VesselName}_CrossSection_{cs_loc}.vtp"
CS_Grd_FileName = f"{VesselName}_CrossSection_{cs_loc}_Grd.vtp"

WriteVTPFile(os.path.join(OutputFolder,CS_FileName), CrossSection)
WriteVTPFile(os.path.join(OutputFolder,CS_Grd_FileName), CrossSectionGrd)

### Cut the Centerline at the Stenosis Location

In [26]:
cs_loc = 50
PathFile_vtp = f"{os.path.splitext(pathline_file)[0]}.vtp"
print(PathFile_vtp)
CenterLine = ReadVTPFile(PathFile_vtp)

plane=vtk.vtkPlane()
plane.SetOrigin(path_points[cs_loc])
plane.SetNormal(direction_points[cs_loc])

clipper = vtk.vtkClipDataSet()
clipper.SetInputData(CenterLine.GetOutput()) 
clipper.SetClipFunction(plane)
clipper.GetOutputInformation(1)
clipper.Update()
cl_path_pre = os.path.join(OutputFolder,f"{VesselName}_PreStenosis.vtp")
WriteVTPFile(cl_path_pre,ExtractSurface(clipper.GetOutput()))

clipper.InsideOutOn()
clipper.Update()
cl_path_post = os.path.join(OutputFolder,f"{VesselName}_PostStenosis.vtp")
WriteVTPFile(cl_path_post,ExtractSurface(clipper.GetOutput()))

/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU07A/SimVascular_Derrick/Paths/L_LAD_0.vtp
